In [16]:
from torch.utils.data import DataLoader
import torch.backends.cudnn as cudnn
import numpy as np

from torchvision.datasets import CIFAR10
from torch.optim.lr_scheduler import CosineAnnealingLR, LinearLR, SequentialLR
import torchvision.transforms as transforms
import os
import pickle
from time import time
from torchsummary import summary
from tqdm.auto import tqdm
import torch

from resnet import ResNet34

In [17]:
device = 'cuda'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
	transforms.RandomCrop(32, padding=4),
	transforms.RandomHorizontalFlip(),
	transforms.ToTensor(),
	transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
	transforms.ToTensor(),
	transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

trainset_nocrop_noflip = CIFAR10(root='./data', train=True, download=True, transform=transform_test)
trainloader_for_Jacobian  = DataLoader(trainset_nocrop_noflip, batch_size=1, shuffle=False, num_workers=2)

sampler=np.random.choice(np.arange(0, 50000), 45000, replace=False)
trainloader_RandomReduced = DataLoader(trainset, batch_size=128, sampler=sampler, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [18]:
# Model
print('==> Building model..')
net = ResNet34()
net = net.to(device)
if device == 'cuda':
	net = torch.nn.DataParallel(net)
	cudnn.benchmark = True


==> Building model..


In [6]:
summary(net,(3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
            Conv2d-3           [-1, 64, 32, 32]          36,864
       BatchNorm2d-4           [-1, 64, 32, 32]             128
            Conv2d-5           [-1, 64, 32, 32]          36,864
       BatchNorm2d-6           [-1, 64, 32, 32]             128
        BasicBlock-7           [-1, 64, 32, 32]               0
            Conv2d-8           [-1, 64, 32, 32]          36,864
       BatchNorm2d-9           [-1, 64, 32, 32]             128
           Conv2d-10           [-1, 64, 32, 32]          36,864
      BatchNorm2d-11           [-1, 64, 32, 32]             128
       BasicBlock-12           [-1, 64, 32, 32]               0
           Conv2d-13           [-1, 64, 32, 32]          36,864
      BatchNorm2d-14           [-1, 64,

In [7]:
# print('==> Resuming from checkpoint..')
# #assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
# checkpoint = torch.load('./ckpt_afterfix_50epoch_0.001reg.pth')
# net.load_state_dict(checkpoint['net'])
# best_acc = checkpoint['acc']
# start_epoch = checkpoint['epoch']

In [15]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)
scheduler1  = CosineAnnealingLR(optimizer, T_max=20000)
scheduler2 = LinearLR(optimizer, start_factor=0.0001, end_factor=1.0, total_iters=2500)
scheduler = SequentialLR(optimizer, schedulers=[scheduler1, scheduler2], milestones=[2500])

def train(epoch):
	print('\nEpoch: %d' % epoch)
	net.train()
	train_loss = 0
	correct = 0
	total = 0
	no_of_training_steps = 0
	for batch_idx, (inputs, targets) in enumerate(trainloader):
			inputs, targets = inputs.to(device), targets.to(device)
			optimizer.zero_grad()
			outputs = net(inputs)
			loss = criterion(outputs, targets)
			loss.backward()
			optimizer.step()

			train_loss += loss.item()
			_, predicted = outputs.max(1)
			total += targets.size(0)
			correct += predicted.eq(targets).sum().item()
			no_of_training_steps = no_of_training_steps + 1

			scheduler.step()
	print(f'Epoch {epoch + 1} -- accuracy: {correct/total:.5f}, num_correct: {correct}, total: {total}, training steps: {no_of_training_steps}')
	#progress_bar(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'  % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))

	return 100.*correct/total

def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            #progress_bar(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)' % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Save checkpoint.
    acc = 100.*correct/total

    print('test',100.*correct/total, correct, total)
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt.pth')
        best_acc = acc

    return 100.*correct/total

NameError: name 'CosineAnnealingLR' is not defined

In [13]:
EPOCHS = 30
for epoch in tqdm(range(EPOCHS)):
	train(epoch)

  0%|          | 0/25 [00:00<?, ?it/s]


Epoch: 0
train 23.792 11896 50000 391

Epoch: 1
train 35.458 17729 50000 391

Epoch: 2
train 44.132 22066 50000 391

Epoch: 3
train 51.96 25980 50000 391

Epoch: 4
train 57.858 28929 50000 391

Epoch: 5
train 63.286 31643 50000 391

Epoch: 6


c:\Users\Nico\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\optim\lr_scheduler.py:149: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


train 69.37 34685 50000 391

Epoch: 7
train 72.552 36276 50000 391

Epoch: 8
train 72.594 36297 50000 391

Epoch: 9
train 72.86 36430 50000 391

Epoch: 10
train 73.632 36816 50000 391

Epoch: 11
train 75.07 37535 50000 391

Epoch: 12
train 76.838 38419 50000 391

Epoch: 13
train 79.346 39673 50000 391

Epoch: 14
train 81.074 40537 50000 391

Epoch: 15
train 82.526 41263 50000 391

Epoch: 16
train 83.814 41907 50000 391

Epoch: 17
train 85.124 42562 50000 391

Epoch: 18
train 85.836 42918 50000 391

Epoch: 19
train 86.72 43360 50000 391

Epoch: 20
train 87.51 43755 50000 391

Epoch: 21
train 88.034 44017 50000 391

Epoch: 22
train 88.652 44326 50000 391

Epoch: 23
train 89.274 44637 50000 391

Epoch: 24
train 89.728 44864 50000 391


In [ ]:
for epoch in tqdm(range(25, 30)):
	train(epoch)

In [14]:
test(-1)

test 84.71 8471 10000
Saving..


84.71

In [ ]:
JacobianList = []
TargetList = []
for inputs, labels in tqdm(trainloader_for_Jacobian):
  TargetList.append(labels)
  start = time()
  jacobian = torch.autograd.functional.jacobian(lambda x: criterion(net(x), labels.to(device)), inputs.to(device))
  JacobianList.append(np.linalg.norm(jacobian.to("cpu")))

In [ ]:
# HessianNormList = []
# TargetList = []
# idx_list = []
# for i, (inputs, labels) in tqdm(enumerate(trainloader_forHessian)):
#   idx_list.append(i)
#   TargetList.append(labels)
#   start = time()
#   hessian_comp = hessian(net, criterion, data=(inputs, labels))
#   top_eigenvalues, top_eigenvector = hessian_comp.eigenvalues(top_n=1)
#   #print("pyhessian : " , top_eigenvalues  , "time : ", (time() - start)         , " seconds "  )
#   HessianNormList.append( top_eigenvalues )

0it [00:00, ?it/s]

In [ ]:
np.array(JacobianList).shape

(50000, 1)

In [ ]:
name = 'JacobianNormList_0.005' #'WideResNet-70-16'
fileObject = open(name, 'wb')
pickle.dump(JacobianList, fileObject)
fileObject.close()

In [ ]:
name = 'TargetList_0.005' #'WideResNet-70-16'
fileObject = open(name, 'wb')
pickle.dump(np.array([np.array(x) for x in TargetList]), fileObject)
fileObject.close()